In [32]:
import pandas as pd
import flair
from dateutil import parser
import nltk 
from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from datetime import timedelta
import numpy as np
pd.options.mode.chained_assignment = None

In [2]:
#Import CSV for tweets
tweet_df = pd.read_csv ('API Data/elon_tweets.csv')
#Load sentiment for flair
flair_sentiment = flair.models.TextClassifier.load('en-sentiment')

2021-07-13 19:06:50,830 loading file C:\Users\Steambun\.flair\models\sentiment-en-mix-distillbert_4.pt


In [3]:
#Create a copy of the tweet df
tweet_processed = tweet_df[['time', 'tweet']].copy()

In [4]:
#Download vader_lexicon if doesn't already exist
#nltk.download('vader_lexicon')

#Declare classifier for NLTK
sid = SentimentIntensityAnalyzer()

#Create list of terms that will be searched within Tweet to determine if the Tweet was referring to DOGE or BTC
BTC_terms = ['bitcoin','btc','crypto','blockchain', 'NFT']
DOGE_terms = ['dog','doge', 'crypto', 'shiba','blockchain']

In [5]:
#Create columns for new DF
tweet_processed['reformat_time'] =''
tweet_processed['flair_sentiment'] =''
tweet_processed['textblob_sentiment'] =''
tweet_processed['nltk_sentiment'] =''
tweet_processed['BTC_in_tweet'] = 0
tweet_processed['DOGE_in_tweet'] = 0

#For loop to go through each tweet one by one
for i in range(len(tweet_processed)):
  
  #Check if Tweet contains BTC or DOGE
    if any(ele in tweet_processed['tweet'][i].lower() for ele in BTC_terms):
        tweet_processed['BTC_in_tweet'][i] = 1
    else:
        pass
    if any(ele in tweet_processed['tweet'][i].lower() for ele in DOGE_terms):
        tweet_processed['DOGE_in_tweet'][i] = 1
    else:
        pass
  
  

  #Flair sentiment
    s = flair.data.Sentence(tweet_processed['tweet'][i])
    flair_sentiment.predict(s)
    total_sentiment = s.labels
    if total_sentiment[0].value == 'NEGATIVE':
        tweet_processed['flair_sentiment'][i] = -total_sentiment[0].score
    else:
        tweet_processed['flair_sentiment'][i] = total_sentiment[0].score

  #NLTK sentiment
    tweet_processed['nltk_sentiment'][i] = sid.polarity_scores(tweet_processed['tweet'][i])['compound']

  #TextBlob sentiment
    tweet_processed['textblob_sentiment'][i] = TextBlob(tweet_processed['tweet'][i]).sentiment.polarity

  #Reformat time
    date = parser.parse(tweet_processed['time'][i])
    #Remove seconds and minutes as we want to round next hour
    date = date.replace(second=0, minute=0)
    #Round to the next hour as Tweets affect future price times 
    date = date + timedelta(hours = 1)
    tweet_processed['reformat_time'][i] = date.strftime("%Y-%m-%d %H:%M:%S")

<ipython-input-5-8eb8629c0dcb>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweet_processed['flair_sentiment'][i] = total_sentiment[0].score
<ipython-input-5-8eb8629c0dcb>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweet_processed['nltk_sentiment'][i] = sid.polarity_scores(tweet_processed['tweet'][i])['compound']
<ipython-input-5-8eb8629c0dcb>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweet_processed['textblob_senti

In [6]:
#tweet_processed.to_csv('Raw Processed Tweets.csv')

In [14]:
tweet_processed

,time,tweet,reformat_time,flair_sentiment,textblob_sentiment,nltk_sentiment,BTC_in_tweet,DOGE_in_tweet
0,2021-07-07T18:48:11.000Z,Supersonic shock wave,2021-07-07 19:00:00,0.939953,0.0,-0.3818,0,0
1,2021-07-07T18:46:53.000Z,https://t.co/O3BGaFuMDO,2021-07-07 19:00:00,-0.708477,0.0,0.0,0,0
2,2021-07-07T16:58:11.000Z,High res landing Falcon landing video https://...,2021-07-07 17:00:00,0.99812,0.16,0.0,0,0
3,2021-07-06T22:09:32.000Z,https://t.co/TQZ9QvcwH1,2021-07-06 23:00:00,0.855198,0.0,0.0,0,0
4,2021-07-05T07:12:04.000Z,Free Britney,2021-07-05 08:00:00,-0.73463,0.4,0.5106,0,0
...,...,...,...,...,...,...,...,...
629,2020-06-10T18:22:29.000Z,"Moreover, energy produced by Tesla solar deplo...",2020-06-10 19:00:00,0.897086,0.1,0.4939,0,0
630,2020-06-10T18:21:05.000Z,Electric cars produce *much* less CO2 than fos...,2020-06-10 19:00:00,-0.681038,-0.166667,0.0,0,0
631,2020-06-10T18:07:52.000Z,https://t.co/QPwGa1Rk8Z,2020-06-10 19:00:00,0.882915,0.0,0.0,0,0
632,2020-06-10T18:07:22.000Z,"Posting Tesla Impact Report, this time with su...",2020-06-10 19:00:00,-0.536929,0.0,0.0,0,0


In [30]:
tweet_raw = pd.read_csv('Raw Processed Tweets.csv',index_col = 0)

In [29]:
tweet_feature = tweet_raw[['reformat_time','flair_sentiment', 'textblob_sentiment', 'nltk_sentiment', 'BTC_in_tweet', 'DOGE_in_tweet']]
tweet_pivot = pd.pivot_table(tweet_feature, values = ['flair_sentiment', 'textblob_sentiment', 'nltk_sentiment', 'BTC_in_tweet', 'DOGE_in_tweet'], index = ['reformat_time'], aggfunc = np.mean)
tweet_pivot = tweet_pivot.reset_index()
tweet_pivot

,reformat_time,BTC_in_tweet,DOGE_in_tweet,flair_sentiment,nltk_sentiment,textblob_sentiment
0,2020-06-10 15:00:00,0.0,0.0,-0.990820,0.000000,0.200000
1,2020-06-10 19:00:00,0.0,0.0,0.093493,0.082317,0.005556
2,2020-06-10 20:00:00,0.0,0.0,0.968325,0.817600,0.183333
3,2020-06-10 23:00:00,0.0,0.0,0.980122,0.624900,0.500000
4,2020-06-11 10:00:00,0.0,0.0,0.737393,0.421500,0.800000
...,...,...,...,...,...,...
464,2021-07-04 19:00:00,0.0,0.0,0.997125,0.611400,1.000000
465,2021-07-05 08:00:00,0.0,0.0,-0.734630,0.510600,0.400000
466,2021-07-06 23:00:00,0.0,0.0,0.855198,0.000000,0.000000
467,2021-07-07 17:00:00,0.0,0.0,0.998120,0.000000,0.160000


In [45]:
tweet_doge = tweet_pivot[['reformat_time']]
tweet_doge['sentiment'] = (tweet_pivot.flair_sentiment + tweet_pivot.nltk_sentiment + tweet_pivot.textblob_sentiment)/3*tweet_pivot.DOGE_in_tweet
for i in range(len(tweet_doge)):
    if tweet_doge.sentiment[i] > 0:
        tweet_doge.sentiment[i] = 1
    elif tweet_doge.sentiment[i] < 0:
        tweet_doge.sentiment[i] = -1 
    else:
        tweet_doge.sentiment[i] = 0
tweet_doge = tweet_doge.rename(columns = {'reformat_time':'time'})

In [57]:
tweet_doge

,time,sentiment
0,2020-06-10 15:00:00,0.0
1,2020-06-10 19:00:00,0.0
2,2020-06-10 20:00:00,0.0
3,2020-06-10 23:00:00,0.0
4,2020-06-11 10:00:00,0.0
...,...,...
464,2021-07-04 19:00:00,0.0
465,2021-07-05 08:00:00,0.0
466,2021-07-06 23:00:00,0.0
467,2021-07-07 17:00:00,0.0


In [47]:
tweet_btc = tweet_pivot[['reformat_time']]
tweet_btc['sentiment'] = (tweet_pivot.flair_sentiment + tweet_pivot.nltk_sentiment + tweet_pivot.textblob_sentiment)/3*tweet_pivot.BTC_in_tweet
for i in range(len(tweet_btc)):
    if tweet_btc.sentiment[i] > 0:
        tweet_btc.sentiment[i] = 1
    elif tweet_btc.sentiment[i] < 0:
        tweet_btc.sentiment[i] = -1 
    else:
        tweet_btc.sentiment[i] = 0
tweet_btc = tweet_btc.rename(columns = {'reformat_time':'time'})

In [56]:
tweet_btc

,time,sentiment
0,2020-06-10 15:00:00,0.0
1,2020-06-10 19:00:00,0.0
2,2020-06-10 20:00:00,0.0
3,2020-06-10 23:00:00,0.0
4,2020-06-11 10:00:00,0.0
...,...,...
464,2021-07-04 19:00:00,0.0
465,2021-07-05 08:00:00,0.0
466,2021-07-06 23:00:00,0.0
467,2021-07-07 17:00:00,0.0


In [49]:
tweet_doge.to_csv('DOGE Sentiment.csv')
tweet_btc.to_csv('BTC Sentiment.csv')